# LangChain Expression Language (LCEL) Tutorial

In [3]:
from typing import Any, Dict, List

from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser

# Import LangChain components
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_ollama import ChatOllama
from rich import print

In [4]:
# Load environment variables
load_dotenv()

# Initialize the LLM
llm = ChatOllama(model="llama3.2",temperature=0.3)

## SECTION 1: Basic LCEL Chain

In [5]:
print("-" * 50,'\n',"DEMO 1: Basic LCEL Chain",'\n',"-" * 50)

# Create a simple prompt template
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}.")

# Create a basic chain using the | operator (pipe)
basic_chain = prompt | llm | StrOutputParser()


print("Prompt: ", prompt.format_prompt(topic="cats").to_messages())
print("Chain: ", basic_chain.invoke({"topic": "cats"}))

-------------------------------------------------- 
 DEMO 1: Basic LCEL Chain 
 --------------------------------------------------

Prompt: 
[HumanMessage(content='Tell me a short joke about cats.', additional_kwargs={}, response_metadata={})]

Chain:  Why did the cat join a band?

Because it wanted to be the purr-cussionist!

In [6]:
# Run the chain
result = basic_chain.invoke({"topic": "programming"})
print(result)
print("\n")

Why do programmers prefer dark mode?

Because light attracts bugs.

## SECTION 2: Parallel Processing with LCEL

In [7]:
print("-" * 50,'\n',"DEMO 2: Parallel Processing with LCEL",'\n',"-" * 50)

# Create two different prompts
pros_prompt = ChatPromptTemplate.from_template(
    "List 3 pros of {technology} technology."
)
cons_prompt = ChatPromptTemplate.from_template(
    "List 3 cons of {technology} technology."
)

# Create chains for each prompt
pros_chain = pros_prompt | llm | StrOutputParser()
cons_chain = cons_prompt | llm | StrOutputParser()

# Create a parallel chain
parallel_chain = RunnableParallel(pros=pros_chain, cons=cons_chain)

print("Prompt: ", pros_prompt.format_prompt(technology="AI").to_messages())
print("Chain: ", parallel_chain.invoke({"technology": "AI"}))

-------------------------------------------------- 
 DEMO 2: Parallel Processing with LCEL 
 --------------------------------------------------

Prompt: 
[HumanMessage(content='List 3 pros of AI technology.', additional_kwargs={}, response_metadata={})]

Chain: 
{
    'pros': "Here are three pros of AI technology:\n\n1. **Improved Efficiency**: Artificial intelligence can 
automate repetitive and mundane tasks, freeing up human time and energy for more strategic and creative work. This 
can lead to increased productivity and efficiency in various industries such as healthcare, finance, and customer 
service.\n\n2. **Enhanced Decision-Making**: AI algorithms can analyze vast amounts of data quickly and accurately,
providing insights that may not be apparent to humans. This can help organizations make better decisions, predict 
trends, and identify opportunities for growth.\n\n3. **Personalized Experiences**: AI-powered systems can learn 
about individual preferences and behaviors, allowing for personalized experiences in areas such as customer 
service, marketing, and education. For example, AI-powered chatbots can offer tailored support to customers, while 
AI-driven recommendation engines can suggest products or services based on a user's interests.\n\nThese are just a 
few examples of the many benefits that AI technology can bring.",
    'cons': "Here are three cons of AI technology:\n\n1. **Job Displacement**: One of the significant concerns with
AI is its potential to automate jobs, leading to widespread unemployment and displacement of workers. As machines 
become more efficient and capable, they may replace human workers in various industries, including manufacturing, 
customer service, and other sectors.\n\n2. **Bias and Discrimination**: AI systems can perpetuate existing biases 
and discriminatory practices if they are trained on biased data or designed with a particular worldview. This can 
lead to unfair treatment of certain groups, such as racial minorities, women, or individuals with 
disabilities.\n\n3. **Loss of Human Touch and Empathy**: Over-reliance on AI technology can lead to a loss of human
touch and empathy in various aspects of life, including customer service, healthcare, and education. While AI can 
provide efficient and accurate information, it often lacks the emotional intelligence and compassion that humans 
bring to these interactions.\n\nThese are just a few examples of the potential downsides of AI technology. However,
it's worth noting that many experts believe that the benefits of AI far outweigh its drawbacks, and that 
responsible development and deployment of AI can help mitigate some of its negative consequences."
}

In [8]:

# Run the parallel chain
parallel_result = parallel_chain.invoke({"technology": "blockchain"})
print("PROS:")
print(parallel_result["pros"])
print("\nCONS:")
print(parallel_result["cons"])
print("\n")

PROS:

Here are three pros of blockchain technology:

1. **Security and Transparency**: Blockchain technology uses a decentralized, distributed ledger system that 
records transactions across multiple nodes on a network. This makes it virtually impossible to alter or manipulate 
data, ensuring the integrity and security of transactions.

2. **Immutable Record Keeping**: The use of blockchain technology allows for an immutable record keeping system, 
where once a transaction is recorded, it cannot be altered or deleted. This provides a permanent and tamper-proof 
record of all transactions, which can help build trust among participants in a network.

3. **Decentralized and Autonomous**: Blockchain technology operates on a decentralized network, meaning that no 
single entity controls the data or transactions. This allows for greater autonomy and decision-making power to be 
distributed among participants, rather than being controlled by a central authority.

CONS:

Here are three cons of blockchain technology:

1. **Scalability Limitations**: Blockchain technology is still in its early stages, and many blockchains struggle 
to scale to meet the demands of widespread adoption. As more users join a network, it can become increasingly 
difficult for transactions to be processed quickly and efficiently.

2. **Energy Consumption**: The process of validating transactions on a blockchain requires significant 
computational power, which often results in high energy consumption. This has raised concerns about the 
environmental impact of blockchain technology, particularly with regards to cryptocurrencies like Bitcoin.

3. **Regulatory Uncertainty**: Blockchain technology operates outside of traditional regulatory frameworks, which 
can create uncertainty and confusion for businesses and individuals looking to use it. Different countries have 
varying laws and regulations regarding blockchain, and it's often unclear what rules apply in different 
jurisdictions.

## SECTION 3: Advanced Chain with Data Transformation

In [9]:
print("-" * 50,'\n',"DEMO 3: Advanced Chain with Data Transformation",'\n',"-" * 50)

# Define a function to process input
def preprocess_input(input_data: Dict[str, Any]) -> Dict[str, Any]:
    """Process the input data before sending to the LLM."""
    # Add a timestamp, capitalize the query, etc.
    processed_data = input_data.copy()
    processed_data["query"] = input_data["query"].upper()
    processed_data["word_count"] = len(input_data["query"].split())
    return processed_data


# Define a function to process output
def postprocess_output(output: str) -> Dict[str, Any]:
    """Process the output from the LLM."""
    lines = output.strip().split("\n")
    return {
        "summary": lines[0] if lines else "",
        "details": lines[1:],
        "response_length": len(output),
    }


# Create an advanced prompt
advanced_prompt = ChatPromptTemplate.from_template(
    """Answer the following query: {query}

    The query has {word_count} words.
    Provide a detailed explanation.
    """
)

# Create an advanced chain with preprocessing and postprocessing
advanced_chain = (
    RunnableLambda(preprocess_input)
    | advanced_prompt
    | llm
    | StrOutputParser()
    | RunnableLambda(postprocess_output)
)

# Preprocess the input data
preprocessed_data = preprocess_input({"query": "Explain quantum computing"})

print("Prompt: ", advanced_prompt.format_prompt(**preprocessed_data).to_messages())
print("Chain: ", advanced_chain.invoke(preprocessed_data))

-------------------------------------------------- 
 DEMO 3: Advanced Chain with Data Transformation 
 --------------------------------------------------

Prompt: 
[
    HumanMessage(
        content='Answer the following query: EXPLAIN QUANTUM COMPUTING\n\n    The query has 3 words.\n    Provide a
detailed explanation.\n    ',
        additional_kwargs={},
        response_metadata={}
    )
]

Chain: 
{
    'summary': "Quantum Computing is a revolutionary technology that uses the principles of quantum mechanics to 
perform calculations and operations on data. It's a field that has gained significant attention in recent years due
to its potential to solve complex problems that are currently unsolvable or require an unfeasible amount of time 
using classical computers.",
    'details': [
        '',
        '**What is Quantum Computing?**',
        '',
        'Classical computers use bits, which can have a value of either 0 or 1. In contrast, quantum computers use 
qubits (quantum bits), which can exist in multiple states simultaneously, known as superposition. This property 
allows qubits to process multiple possibilities at the same time, making them much faster than classical computers 
for certain types of calculations.',
        '',
        '**Key Principles of Quantum Computing**',
        '',
        '1. **Superposition**: Qubits can exist in multiple states (0, 1, or both) at the same time.',
        "2. **Entanglement**: Qubits can be connected in such a way that their properties are correlated, even if 
they're separated by large distances.",
        '3. **Quantum Measurement**: When a qubit is measured, its state collapses to one of the possible outcomes 
(0 or 1).',
        '4. **Quantum Interference**: The superposition of qubits can lead to interference patterns, which can be 
exploited for quantum computing.',
        '',
        '**How Does Quantum Computing Work?**',
        '',
        'A quantum computer consists of several components:',
        '',
        '1. **Qubits**: These are the basic units of quantum information.',
        '2. **Quantum Gates**: These are the quantum equivalent of logic gates in classical computers. They perform
operations on qubits, such as rotation and entanglement.',
        '3. **Quantum Algorithms**: These are the programs that run on a quantum computer. They use quantum 
mechanics to solve specific problems.',
        '',
        '**Types of Quantum Computing**',
        '',
        '1. **Gate-Based Quantum Computing**: This is the most common type, where quantum gates are used to 
manipulate qubits.',
        '2. **Analog Quantum Computing**: This approach uses continuous-variable quantum systems, such as 
superconducting circuits or optical systems.',
        '3. **Topological Quantum Computing**: This method uses exotic materials called topological insulators to 
create a robust and fault-tolerant quantum computer.',
        '',
        '**Applications of Quantum Computing**',
        '',
        '1. **Cryptography**: Quantum computers can break certain classical encryption algorithms, but they also 
enable new, quantum-resistant cryptography methods.',
        '2. **Optimization**: Quantum computers can solve complex optimization problems much faster than classical 
computers.',
        '3. **Simulation**: Quantum computers can simulate complex systems, such as molecules and chemical 
reactions, which could lead to breakthroughs in fields like chemistry and materials science.',
        '4. **Machine Learning**: Quantum computers can speed up certain machine learning algorithms, leading to 
potential improvements in areas like image recognition and natural language processing.',
        '',
        '**Challenges and Limitations**',
        '',
        '1. **Error Correction**: Quantum computers are prone to errors due to the fragile nature of qubits. 
Developing robust error correction techniques is essential.',
        '2. **Scalability**: Currently, most quantum computers are small-scale and need to be scaled up to perform 
practical calculations.',
        '3. **Quantum Noise**: Quantum systems are sensitive to noise, which can cause errors in computations.',
        '',
        '**Conclusion**',
        '',
        'Quantum Computing has the potential to revolutionize various fields by solving complex problems that are 
currently unsolvable or require an unfea

In [10]:
# Run the advanced chain
advanced_result = advanced_chain.invoke(
    {"query": "how does LCEL improve LLM applications"}
)
print(f"Summary: {advanced_result['summary']}")
print("Details:")
for detail in advanced_result["details"]:
    if detail.strip():
        print(f"- {detail.strip()}")
print(f"Response Length: {advanced_result['response_length']} characters")
print("\n")

Summary: LCE (Long Context Encoding) is a technique used to improve Large Language Model (LLM) applications. Here's
a detailed explanation of how LCE improves LLM applications:

Details:

- **Background**

- Large Language Models (LLMs) are trained on vast amounts of text data and generate responses based on patterns 
learned from this training data. However, LLMs often struggle with tasks that require understanding the context of 
a conversation or passage, such as answering questions, summarizing text, or generating coherent responses.

- **The Problem**

...

- 1. **Conversational AI**: LCE improves conversational AI systems by enabling them to understand context and 
generate more coherent responses.

- 2. **Text Summarization**: LCE enhances text summarization models by allowing them to capture more context 
information and generate more concise summaries.

- 3. **Question Answering**: LCE improves question answering models by enabling them to understand the context of 
the input prompt and generate more accurate answers.

- In summary, LCE is a technique that uses Long Context Encoding to improve Large Language Model applications by 
capturing more context information and generating more coherent and consistent responses.

Response Length: 3093 characters

## SECTION 4: RAG Pattern with LCEL

In [11]:
print("-" * 50,'\n',"DEMO 4: RAG Pattern with LCEL (Simulated)",'\n',"-" * 50)


# Simulate a retriever function
def simulate_retrieval(query: str) -> List[str]:
    """Simulate document retrieval based on query."""
    # In a real application, this would query a vector database
    documents = [
        "LCEL allows for declarative chain construction in LangChain.",
        "LangChain Expression Language simplifies building complex LLM applications.",
        "LCEL supports streaming, async operations, and parallel execution.",
    ]
    return documents


# Create a RAG prompt
rag_prompt = ChatPromptTemplate.from_template(
    """Answer the question based on the following context:

    Context:
    {context}

    Question: {question}
    """
)


# Create a RAG chain
def format_docs(docs):
    return "\n".join(docs)


rag_chain = (
    RunnableParallel(
        {
            "context": RunnableLambda(simulate_retrieval) | RunnableLambda(format_docs),
            "question": RunnablePassthrough(),
        }
    )
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("Prompt: ", rag_prompt.format_prompt(context="context", question="question").to_messages())
print("Chain: ", rag_chain.invoke({"question": "What is LCEL?"}))

-------------------------------------------------- 
 DEMO 4: RAG Pattern with LCEL (Simulated) 
 --------------------------------------------------

Prompt: 
[
    HumanMessage(
        content='Answer the question based on the following context:\n\n    Context:\n    context\n\n    Question: 
question\n    ',
        additional_kwargs={},
        response_metadata={}
    )
]

Chain:  Based on the provided context, LCEL stands for LangChain Expression Language. It is a simplification of 
building complex Large Language Model (LLM) applications in LangChain, allowing for declarative chain construction 
and supporting features such as streaming, async operations, and parallel execution.

In [12]:
# Run the RAG chain
rag_result = rag_chain.invoke("What is LCEL and why is it useful?")
print(rag_result)

Based on the provided context, LCEL stands for LangChain Expression Language. It is a tool that allows for 
declarative chain construction in LangChain, which simplifies building complex Large Language Model (LLM) 
applications. LCEL is useful because it supports streaming, async operations, and parallel execution, making it 
easier to construct complex chains of operations and improve the performance of LLM applications.